In [1]:
from mongoengine import *
from mongoengine.fields import *
from settings import db_config
import model.restaurant as r_model

In [30]:
class RestaurantQuerySet(QuerySet):
    def search_by_avgCost(self, address, min_cost=0, max_cost=100, meter = 500):
        return self.filter(Q(avgCost__in=range(min_cost, max_cost+1)) 
                           & Q(address__near=list(address), address__max_distance=meter))
    def search_by_name(self, name):
        return self.filter(name=name)
    def search_by_id(self, rid):
        return self.filter(rid=rid)
    def search_by_address(self, address, order_by='', meter = 500):
        return self.filter(address__near=list(location), address__max_distance=meter)
    def search_by_catelog(self, address, catelog, meter = 500):
        return self.filter(Q(classify=catelog)
                            & Q(address__near=list(address), address__max_distance=meter))
    def list_cate(self):
        return self.distinct('classify')
class Restaurant(Document):
    meta = {
        'collection': 'restaurant',
        'queryset_class': RestaurantQuerySet,
        'strict': False,
    }
    environmentRate = FloatField(min_value=0)
    serviceRate = FloatField(min_value=0)
    tasteRate = FloatField(min_value=0)
    classify = StringField(required=True)
    avgCost = IntField(min_value=0)
    address = StringField(required=True)
    hours = ListField()
    name = StringField(required=True)
    rid = LongField(required=True)

In [3]:
def string_to_point(str_point):
    try:
        point = list(reversed([float(x) for x in str_point.split(',')]))
    except ValueError:
        print(str_point, '---------------------------value error------------------------')
        point = None
    except AttributeError:
        print(str_point, '---------------------------attribute error------------------------')
        point = None
    return point

In [4]:
# update address to point type
for i in Restaurant.objects:
    for r in Restaurant.objects(rid=i.rid):
        point = string_to_point(r.address)
        if point is not None:
            print(point)
            i.update(address=point)

In [5]:
# reverse list to [longitude, latitude]
for i in Restaurant.objects:
    tmp = i.address['coordinates']
    tmp = list(reversed(tmp))
    i.update(address=tmp)

In [36]:
# for i in Restaurant.objects:
#     print(i..search_by_address((121.56008359778, 25.080193176667)))
for i in r_model.Restaurant.objects.search_by_address(location=(121.56008359778, 25.080193176667), 
                                                     meter=100):
    print(type(i.avgCost))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [37]:
# for i in Restaurant.objects.search_by_catelog((121.56008359778, 25.080193176667), cate, meter=500):
#     print(i.classify)

NameError: name 'datetime' is not defined